Library imports

In [1]:
import tensorflow as tf
from keras import layers
from keras.datasets import mnist
from keras.layers import Dropout
from tensorflow import keras

import matplotlib.pyplot as plt

2022-04-08 23:19:41.387488: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-08 23:19:41.387516: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Dir for images datasets

In [2]:
import os, shutil, pathlib

original_dir = pathlib.Path("../dataset/intel_images")
new_base_dir = pathlib.Path("../dataset/intel_images_small")

Image Augmentation 

In [3]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

2022-04-08 23:19:51.595781: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-08 23:19:51.595846: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-08 23:19:51.595876: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (linux): /proc/driver/nvidia/version does not exist
2022-04-08 23:19:51.596344: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1. / 255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(units=6, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 180, 180, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                             

In [6]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
    )

/home/avizyt/anaconda3/envs/tflow/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="../models/checkpoints/intel_image_clf_augmentation.keras",
        save_best_only=True,
        monitor='val_accuracy'
    )
]

### Prepare images for training

In [8]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    directory=new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32
)
validation_dataset = image_dataset_from_directory(
    directory=new_base_dir / "valid",
    image_size=(180, 180),
    batch_size=32
)
test_dataset = image_dataset_from_directory(
    directory=new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32
)

Found 6000 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


In [9]:
for data_batch, labels_batch in train_dataset:
    print("Data batch shape: ", data_batch.shape)

    print("Labels batch shape: ", labels_batch.shape)
    break

Data batch shape:  (32, 180, 180, 3)
Labels batch shape:  (32,)


Training the model

In [11]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=callbacks
)

Epoch 1/10
188/188 [==============================] - 292s 2s/step - loss: 1.2230 - accuracy: 0.5108 - val_loss: 1.0454 - val_accuracy: 0.5930
Epoch 2/10
188/188 [==============================] - 312s 2s/step - loss: 1.0355 - accuracy: 0.5972 - val_loss: 0.9127 - val_accuracy: 0.6413
Epoch 3/10
188/188 [==============================] - 304s 2s/step - loss: 0.9488 - accuracy: 0.6417 - val_loss: 0.7905 - val_accuracy: 0.7023
Epoch 4/10
188/188 [==============================] - 288s 2s/step - loss: 0.8670 - accuracy: 0.6720 - val_loss: 0.7946 - val_accuracy: 0.7167
Epoch 5/10
188/188 [==============================] - 287s 2s/step - loss: 0.8142 - accuracy: 0.6947 - val_loss: 0.6639 - val_accuracy: 0.7597
Epoch 6/10
188/188 [==============================] - 292s 2s/step - loss: 0.7676 - accuracy: 0.7135 - val_loss: 0.7629 - val_accuracy: 0.7117
Epoch 7/10
188/188 [==============================] - 299s 2s/step - loss: 0.7344 - accuracy: 0.7225 - val_loss: 0.7506 - val_accuracy: 0.7073

In [12]:
test_model = keras.models.load_model("../models/checkpoints/intel_image_clf_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test Loss: {test_loss},Test Accuracy: {test_acc}")

94/94 [==============================] - 36s 364ms/step - loss: 0.6306 - accuracy: 0.7663
Test Loss: 0.6306498646736145,Test Accuracy: 0.7663333415985107


Model 2

In [18]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1. / 255)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)

# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)

x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)

x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x)

x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=512, kernel_size=3, activation='relu')(x)

x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=1024, kernel_size=3, activation='relu')(x)

x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=1024, kernel_size=3, activation='relu')(x)

x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(units=6, activation='softmax')(x)
model2 = keras.Model(inputs=inputs, outputs=outputs)

In [19]:
model2.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 180, 180, 3)       0         
                                                                 
 rescaling_6 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_34 (Conv2D)          (None, 178, 178, 64)      1792      
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 89, 89, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_35 (Conv2D)          (None, 87, 87, 128)       73856     
                                                           

In [22]:
model2.compile(
    optimizer=keras.optimizers.Adam(lr=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
    )

/home/avizyt/anaconda3/envs/tflow/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [20]:
callbacks_model2 = [
    keras.callbacks.ModelCheckpoint(
        filepath="../models/checkpoints/intel_image_clf_augmentation2.keras",
        save_best_only=True,
        monitor='val_accuracy'
    )
]

In [23]:
history_model2 = model2.fit(
    train_dataset,
    epochs=2,
    validation_data=validation_dataset,
    callbacks=callbacks_model2
)

Epoch 1/2


2022-04-09 00:35:39.219917: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.
2022-04-09 00:35:46.186495: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.


  1/188 [..............................] - ETA: 33:18 - loss: 1.7886 - accuracy: 0.1875

2022-04-09 00:35:48.840277: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.
2022-04-09 00:35:53.228997: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.


  2/188 [..............................] - ETA: 18:32 - loss: 2.0778 - accuracy: 0.1719

2022-04-09 00:35:54.037848: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.


 24/188 [==>...........................] - ETA: 14:04 - loss: 1.8194 - accuracy: 0.1680

KeyboardInterrupt: 